In [ ]:
#recombination_rate=1e-8
#mutation_rate=1e-8, 0.2e-8 , 5e-8

In [1]:
import stdpopsim
import os
import tskit
import msprime
import numpy as np
import tsinfer
import zarr
import demesdraw
import subprocess
import pyfaidx
import tsdate
import matplotlib.pyplot as plt
from IPython.display import display, SVG
from Bio import bgzf
#display(SVG(ts.draw_svg()))
from OOA_sim import simple_OOA_sim

/Users/jinminli/research/tree_distance/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def dump_ts(ts,save_dir="./simulations/sim1/",vcz_dir="./simulations/sim1/",name="test",contig_id=0):
    os.makedirs(save_dir, exist_ok=True)
    ts.dump(save_dir+name+".trees")
    np.save(f"{vcz_dir+name}-AA.npy", [s.ancestral_state for s in ts.sites()])
    vcf_name = f"{vcz_dir+name}.vcf.gz"
    with bgzf.open(vcf_name, "wt") as f:
        ts.write_vcf(f,contig_id=contig_id)
    subprocess.run(["tabix", vcf_name])
    ret = subprocess.run(
        ["python", "-m", "bio2zarr", "vcf2zarr", "convert", "--force", vcf_name, f"{vcz_dir+name}.vcz"],
        stderr = None,
    )
    if ret.returncode != 0: 
        print(f"[ERROR] bio2zarr failed for {name} with code {ret.returncode}") 
    else: 
        print(f"[OK] bio2zarr finished for {name}")
def infer_ts(vcz_full_name,tree_full_name):
    #load zarr file, ancestral states and infer
    ancestral_states = np.load(f"{vcz_full_name}-AA.npy")
    vdata = tsinfer.VariantData(f"{vcz_full_name}.vcz", ancestral_states)
    inferred_ts=tsinfer.infer(vdata,recombination_rate=1e-8)
    inferred_ts.dump(f"{tree_full_name}-inferred.trees")
    print("tsinfer done")
    print("Running tsdate")
    simplified_ts = tsdate.preprocess_ts(inferred_ts)

    redated_ts = tsdate.date(simplified_ts, mutation_rate=1e-8)

    out_file = f"{tree_full_name}-inferred.tsdate.trees"
    redated_ts.dump(out_file)

In [9]:
#specifying rate version
def simulate_dump(model,number_individuals,num_replicates,tree_dir,vcz_dir,sequence_length=1e6,mut_rate=1e-8):
    if isinstance(model, float):
        name = f"Ne{model:.0e}_r{mut_rate:.0e}_{2*number_individuals}_{sequence_length:.0e}"
        
        tree_base_path = os.path.join(tree_dir, name)
        vcz_base_path=os.path.join(vcz_dir, name)
        os.makedirs(tree_base_path, exist_ok=True)
        os.makedirs(vcz_base_path, exist_ok=True)
        for i in range(num_replicates):
            tree_replicate_dir = os.path.join(tree_base_path, str(i))+"/"
            vcz_replicate_dir = os.path.join(vcz_base_path, str(i))+"/"
            os.makedirs(tree_replicate_dir, exist_ok=True)
            os.makedirs(vcz_replicate_dir, exist_ok=True)
            ts=msprime.sim_ancestry(samples=number_individuals,population_size=model,sequence_length=sequence_length,recombination_rate=1e-8)
            ts=msprime.sim_mutations(ts,rate=mut_rate)
            #remove site at 0
            if ts.site(0).position==0:
                ts=ts.delete_sites(0)
            dump_ts(ts,save_dir=tree_replicate_dir,vcz_dir=vcz_replicate_dir,name=name+"_"+str(i))
            infer_ts(vcz_full_name=vcz_replicate_dir+name+"_"+str(i),tree_full_name=tree_replicate_dir+name+"_"+str(i))
    else:
        assert(isinstance(model, str))
        assert(model=="ooa")
        name=f"ooa_r{mut_rate:.0e}_{6*number_individuals}_{sequence_length:.0e}"
    
        tree_base_path = os.path.join(tree_dir, name)
        vcz_base_path=os.path.join(vcz_dir, name)
        os.makedirs(tree_base_path, exist_ok=True)
        os.makedirs(vcz_base_path, exist_ok=True)
        for i in range(num_replicates):
            tree_replicate_dir = os.path.join(tree_base_path, str(i))+"/"
            vcz_replicate_dir = os.path.join(vcz_base_path, str(i))+"/"
            os.makedirs(tree_replicate_dir, exist_ok=True)
            os.makedirs(vcz_replicate_dir, exist_ok=True)
            ts=simple_OOA_sim(number_individuals,sequence_length,mut_rate=mut_rate)
            #remove site at 0
            if ts.site(0).position==0:
                ts=ts.delete_sites(0)
            dump_ts(ts,save_dir=tree_replicate_dir,vcz_dir=vcz_replicate_dir,name=name+"_"+str(i))
            infer_ts(vcz_full_name=vcz_replicate_dir+name+"_"+str(i),tree_full_name=tree_replicate_dir+name+"_"+str(i))

In [10]:
simulate_dump(2e4,50,100,tree_dir="/Users/jinminli/research/tree_distance/simulations/diff_rates/tree/",vcz_dir="/Users/jinminli/research/tree_distance/simulations/diff_rates/vcf/",mut_rate=1e-8)

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 93.4files/s]
 Explode: 100%|██████████| 4.26k/4.26k [00:00<00:00, 23.4kvars/s]
  Encode: 100%|██████████| 1.25M/1.25M [00:00<00:00, 9.67MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 869array/s]


[OK] bio2zarr finished for Ne2e+04_r1e-08_100_1e+06_0
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.0files/s]
 Explode: 100%|██████████| 3.95k/3.95k [00:00<00:00, 20.0kvars/s]
  Encode: 100%|██████████| 1.16M/1.16M [00:00<00:00, 12.5MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 1.08karray/s]


[OK] bio2zarr finished for Ne2e+04_r1e-08_100_1e+06_1
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 84.7files/s]
 Explode: 100%|██████████| 4.32k/4.32k [00:00<00:00, 23.0kvars/s]
  Encode: 100%|██████████| 1.27M/1.27M [00:00<00:00, 11.3MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 1.03karray/s]


...

[OK] bio2zarr finished for Ne2e+04_r1e-08_100_1e+06_97
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 79.0files/s]
 Explode: 100%|██████████| 4.26k/4.26k [00:00<00:00, 22.5kvars/s]
  Encode: 100%|██████████| 1.25M/1.25M [00:00<00:00, 12.1MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 872array/s]


[OK] bio2zarr finished for Ne2e+04_r1e-08_100_1e+06_98
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 79.0files/s]
 Explode: 100%|██████████| 4.45k/4.45k [00:00<00:00, 23.4kvars/s]
  Encode: 100%|██████████| 1.31M/1.31M [00:00<00:00, 11.7MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 873array/s]


[OK] bio2zarr finished for Ne2e+04_r1e-08_100_1e+06_99
tsinfer done
Running tsdate


In [13]:
simulate_dump(2e4,50,100,tree_dir="/Users/jinminli/research/tree_distance/simulations/diff_rates/tree/",vcz_dir="/Users/jinminli/research/tree_distance/simulations/diff_rates/vcf/",mut_rate=5e-8)

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 84.6files/s]
 Explode: 100%|██████████| 20.3k/20.3k [00:00<00:00, 23.0kvars/s]
  Encode: 100%|██████████| 5.96M/5.96M [00:00<00:00, 13.0MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 455array/s]


[OK] bio2zarr finished for Ne2e+04_r5e-08_100_1e+06_0
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.9files/s]
 Explode: 100%|██████████| 20.4k/20.4k [00:00<00:00, 22.9kvars/s]
  Encode: 100%|██████████| 6.00M/6.00M [00:00<00:00, 12.7MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 437array/s]


[OK] bio2zarr finished for Ne2e+04_r5e-08_100_1e+06_1
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.9files/s]
 Explode: 100%|██████████| 21.2k/21.2k [00:00<00:00, 23.6kvars/s]
  Encode: 100%|██████████| 6.23M/6.23M [00:00<00:00, 12.8MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 470array/s]


...

[OK] bio2zarr finished for Ne2e+04_r5e-08_100_1e+06_97
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 83.7files/s]
 Explode: 100%|██████████| 21.2k/21.2k [00:00<00:00, 23.6kvars/s]
  Encode: 100%|██████████| 6.41M/6.41M [00:00<00:00, 13.4MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 399array/s]


[OK] bio2zarr finished for Ne2e+04_r5e-08_100_1e+06_98
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 90.4files/s]
 Explode: 100%|██████████| 20.3k/20.3k [00:00<00:00, 22.6kvars/s]
  Encode: 100%|██████████| 5.96M/5.96M [00:00<00:00, 12.8MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 407array/s]


[OK] bio2zarr finished for Ne2e+04_r5e-08_100_1e+06_99
tsinfer done
Running tsdate


In [14]:
simulate_dump(2e4,50,100,tree_dir="/Users/jinminli/research/tree_distance/simulations/diff_rates/tree/",vcz_dir="/Users/jinminli/research/tree_distance/simulations/diff_rates/vcf/",mut_rate=0.2e-8)

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 79.1files/s]
 Explode: 100%|██████████| 820/820 [00:00<00:00, 17.1kvars/s]
  Encode: 100%|██████████| 235k/235k [00:00<00:00, 8.32MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 870array/s]


[OK] bio2zarr finished for Ne2e+04_r2e-09_100_1e+06_0
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 79.0files/s]
 Explode: 100%|██████████| 781/781 [00:00<00:00, 15.5kvars/s]
  Encode: 100%|██████████| 223k/223k [00:00<00:00, 8.73MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 873array/s]


[OK] bio2zarr finished for Ne2e+04_r2e-09_100_1e+06_1
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.9files/s]
 Explode: 100%|██████████| 839/839 [00:00<00:00, 17.1kvars/s]
  Encode: 100%|██████████| 247k/247k [00:00<00:00, 8.55MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 870array/s]


...

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 82.4files/s]
 Explode: 100%|██████████| 897/897 [00:00<00:00, 17.6kvars/s]
  Encode: 100%|██████████| 257k/257k [00:00<00:00, 6.33MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 907array/s]


[OK] bio2zarr finished for Ne2e+04_r2e-09_100_1e+06_67
tsinfer done
Running tsdate


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 86.3files/s]
 Explode: 100%|██████████| 837/837 [00:00<00:00, 16.9kvars/s]
  Encode: 100%|██████████| 246k/246k [00:00<00:00, 9.22MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 947array/s]


[OK] bio2zarr finished for Ne2e+04_r2e-09_100_1e+06_68
tsinfer done
Running tsdate


AssertionError: Use fewer rescaling intervals